#!pip install --upgrade coremltools

In [1]:
import coremltools
from coremltools.converters import sklearn

dict = {'account':42.11758, 'member':19.31017, 'access':17.05149, 'email':16.6749, 'address':15.3276,
             'update':15.3276, 'price':12.55556, 'market':10.23963, 'online':9.770984, 'information':9.474589,
             'work':8.331161, 'credit':5.883471, 'response':4.504742, 'offer':3.3573, 'transaction':2.809165,
             'agreement':1.863823, 'registration':1.791679, 'person':1.522086, 'system':1.276965,
             'process':1.076627, 'service':0.895862, 'request':0.616316, 'message':0.32805}

key_words = ['account', 'member', 'access', 'email', 'address',
             'update', 'price', 'market', 'online', 'information',
             'work', 'credit', 'response', 'offer', 'transaction',
             'agreement', 'registration', 'person', 'system',
             'process', 'service', 'request', 'message']

# key_words = ['account', 'member', 'access', 'email', 'address',
#              'update', 'price', 'market', 'online', 'information',
#              'work', 'credit', 'response', 'offer', 'transaction',
#              'agreement', 'offer', 'registration']

len(key_words)

23

In [2]:
class MailContentsFeaturize(object):
    def __init__(self, content):
        self.content = str(content).lower()
        
    # 提取特征方法
    def contain_key(self, key):
        if key in self.content:
            # return int(1)
            return float(dict[key])
        else:
            return float(0)
    
    def run(self):
        data = {}
        for word in key_words :
            data[word] = self.contain_key(word)
        return data
    
    def get_train_data(self):
        data = []
        for word in key_words :
            data.append(self.contain_key(word))
        return data

In [3]:
import re, string

def clean_text(text):
    text = text.encode('utf-8').decode('utf-8')
    while '\n' in text:
        text = text.replace('\n', ' ')
    while '  ' in text:
        text = text.replace('  ', ' ')
    words = text.split()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = []
    for token in words: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            stripped.append(new_token.lower())
    text = ' '.join(stripped)
    return text

In [4]:
import os

def get_data(path):
    text_list = list()
    files = os.listdir(path)
    for text_file in files:
        file_path = os.path.join(path, text_file)
        read_file = open(file_path,'r+')
        read_text = read_file.read()
        read_file.close()
        cleaned_text = clean_text(read_text)
        # append text
        # text_list.append(cleaned_text)
        
        featurized = MailContentsFeaturize(cleaned_text)
        # append features
        # text_list.append(featurized.run())
        text_list.append(featurized.get_train_data())
    return text_list, files

In [5]:
# 读取数据
import pandas as pd
train_1_data = pd.read_csv('../output/train_1.csv')
train_2_data = pd.read_csv('../output/train_2.csv')


In [6]:
train_1_pd = pd.DataFrame(train_1_data)
train_1_pd.head()


,Unnamed: 0,0,1
0,0,Looks good to me. Are we certain that all of ...,0
1,1,"Greg, how big was that last check you told me ...",0
2,2,"<table border=0 width=""100%"" cellpadding=""8"" ...",1
3,3,Hello everyone:\n\nIf you were not able to acc...,0
4,4,"I have a customer in town Thursday night, so w...",0


In [7]:
train_1_pd.columns


Index(['Unnamed: 0', '0', '1'], dtype='object')

In [8]:
for word in key_words :
    train_1_pd[word] = train_1_pd["0"].apply(lambda i : MailContentsFeaturize(i).contain_key(word))

train_1_pd.head()

,Unnamed: 0,0,1,account,member,access,email,address,update,price,...,offer,transaction,agreement,registration,person,system,process,service,request,message
0,0,Looks good to me. Are we certain that all of ...,0,0.0,0.0,0.00000,0.0000,0.0000,0.0,12.55556,...,0.0,2.809165,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000
1,1,"Greg, how big was that last check you told me ...",0,0.0,0.0,0.00000,0.0000,0.0000,0.0,0.00000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000
2,2,"<table border=0 width=""100%"" cellpadding=""8"" ...",1,0.0,0.0,0.00000,16.6749,15.3276,0.0,0.00000,...,0.0,2.809165,0.0,0.0,1.522086,0.0,0.0,0.0,0.616316,0.00000
3,3,Hello everyone:\n\nIf you were not able to acc...,0,0.0,0.0,17.05149,0.0000,0.0000,0.0,0.00000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.616316,0.00000
4,4,"I have a customer in town Thursday night, so w...",0,0.0,0.0,0.00000,0.0000,0.0000,0.0,0.00000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.32805


In [9]:
# temp = train_1_pd.drop("0", axis=1)
# temp.to_csv('../output/1.csv')

In [10]:
import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer
# tf_vectorizer = CountVectorizer()
# X = tf_vectorizer.fit_transform(no_head_train)
# print ('#total words', np.matrix.sum(X.todense()))
# print ('#unique words',len(set(tf_vectorizer.get_feature_names())))

In [11]:
# shuffled_indices = np.random.permutation(len(no_head_labels_train))
# train_data = np.array(no_head_train)[shuffled_indices]
# train_data = train_data.tolist()
# train_label = np.array(no_head_labels_train)[shuffled_indices]
# train_label = train_label.tolist()

In [12]:
train_2_pd = pd.DataFrame(train_2_data)
train_2_pd.head()

,Unnamed: 0,0,1
0,0,"Janet,\n\nHere is the model. I have added uni...",0
1,1,"Maureen,\n\nYour presentation:\n\nVince",0
2,2,"Kim, please invite the Following Texas Eastern...",0
3,3,"\n<br>\n\n<p dir=""ltr"" align=""left""><b><font c...",1
4,4,noregrets littlerock2016 alex hoffman democrat...,0


In [13]:
for word in key_words :
    train_2_pd[word] = train_2_pd["0"].apply(lambda i : MailContentsFeaturize(i).contain_key(word))

train_2_pd.head()


,Unnamed: 0,0,1,account,member,access,email,address,update,price,...,offer,transaction,agreement,registration,person,system,process,service,request,message
0,0,"Janet,\n\nHere is the model. I have added uni...",0,0.00000,0.00000,0.0,0.0,0.0000,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,1,"Maureen,\n\nYour presentation:\n\nVince",0,0.00000,0.00000,0.0,0.0,0.0000,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,2,"Kim, please invite the Following Texas Eastern...",0,0.00000,0.00000,0.0,0.0,15.3276,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,3,"\n<br>\n\n<p dir=""ltr"" align=""left""><b><font c...",1,42.11758,19.31017,0.0,0.0,0.0000,15.3276,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.895862,0.0,0.0
4,4,noregrets littlerock2016 alex hoffman democrat...,0,0.00000,0.00000,0.0,0.0,0.0000,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [14]:
train_ = train_1_pd[key_words]
label_ = train_1_pd["1"]
    # .apply(lambda i : int(i))

test_ = train_2_pd[key_words]
label_t = train_2_pd["1"]
    # .apply(lambda i : int(i))

In [15]:
# train_.head()
# label_.head()
# test_.head()
# label_t.head()
# train_.dtypes["account"]
# label_t.dtypes["1"]
label_t.head()

0    0
1    0
2    0
3    1
4    0
Name: 1, dtype: int64

In [16]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression


In [18]:
#Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(train_, label_)

coreml_model = sklearn.convert(dt_model, key_words, 'output')
coreml_model.save('coreml_dt.mlmodel')

dt_predictions = dt_model.predict(test_)
accuracy_score(label_t, dt_predictions)

0.9029022335161161

In [19]:
model = coremltools.models.MLModel('coreml_dt.mlmodel')
# model.predict({'input': temp_test_data})
pred_out_put = model.predict({'account':42.11758, 'member':19.31017, 'access':17.05149, 'email':16.6749, 'address':15.3276,
             'update':15.3276, 'price':12.55556, 'market':10.23963, 'online':9.770984, 'information':9.474589,
             'work':8.331161, 'credit':5.883471, 'response':4.504742, 'offer':3.3573, 'transaction':2.809165,
             'agreement':1.863823, 'registration':1.791679, 'person':1.522086, 'system':1.276965,
             'process':1.076627, 'service':0.895862, 'request':0.616316, 'message':0.32805})
print(pred_out_put["output"])
pred_out_put
# accuracy_score(temp_test_labels,dt_predictions)


0


{'output': 0, 'classProbability': {0: 1.0, 1: 0.0}}

In [20]:
print(confusion_matrix(label_t, dt_predictions))
print(classification_report(label_t, dt_predictions))


[[10113   446]
 [ 1006  3389]]
              precision    recall  f1-score   support

           0       0.91      0.96      0.93     10559
           1       0.88      0.77      0.82      4395

    accuracy                           0.90     14954
   macro avg       0.90      0.86      0.88     14954
weighted avg       0.90      0.90      0.90     14954



In [21]:

#Random Forest
rfc = RandomForestClassifier()
rfc.fit(train_, label_)

coreml_model_rf = sklearn.convert(rfc, key_words, 'output')
coreml_model_rf.save('coreml_rf.mlmodel')

dt_predictions = rfc.predict(test_)
accuracy_score(label_t, dt_predictions)

0.9103918683964157

In [22]:
model = coremltools.models.MLModel('coreml_rf.mlmodel')
# model.predict({'input': temp_test_data})
pred_out_put = model.predict({'account':42.11758, 'member':19.31017, 'access':17.05149, 'email':16.6749, 'address':15.3276,
             'update':15.3276, 'price':12.55556, 'market':10.23963, 'online':9.770984, 'information':9.474589,
             'work':8.331161, 'credit':5.883471, 'response':4.504742, 'offer':3.3573, 'transaction':2.809165,
             'agreement':1.863823, 'registration':1.791679, 'person':1.522086, 'system':1.276965,
             'process':1.076627, 'service':0.895862, 'request':0.616316, 'message':0.32805})
print(pred_out_put["output"])
pred_out_put
# accuracy_score(temp_test_labels,dt_predictions)

0


{'output': 0, 'classProbability': {0: 1.0, 1: 0.0}}

In [23]:
print("Random Forest")
print(confusion_matrix(label_t, dt_predictions))
print(classification_report(label_t, dt_predictions))

Random Forest
[[10171   388]
 [  952  3443]]
              precision    recall  f1-score   support

           0       0.91      0.96      0.94     10559
           1       0.90      0.78      0.84      4395

    accuracy                           0.91     14954
   macro avg       0.91      0.87      0.89     14954
weighted avg       0.91      0.91      0.91     14954



In [24]:
#Logistic Regression
log_model = LogisticRegression()
log_model.fit(train_, label_)

log_predictions = log_model.predict(test_)
accuracy_score(label_t, log_predictions)

/Users/lidayuan/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8620436003744817

In [25]:
print("Logistic Regression")

print(confusion_matrix(label_t, log_predictions))

print(classification_report(label_t, log_predictions))


Logistic Regression
[[9935  624]
 [1439 2956]]
              precision    recall  f1-score   support

           0       0.87      0.94      0.91     10559
           1       0.83      0.67      0.74      4395

    accuracy                           0.86     14954
   macro avg       0.85      0.81      0.82     14954
weighted avg       0.86      0.86      0.86     14954



In [26]:
from sklearn import tree

###create decision tree classifier object
DT = tree.DecisionTreeClassifier(criterion="gini", max_depth=4)
##fit decision tree model with training data
DT.fit(train_, label_)
##test data prediction
DT_expost_preds = DT.predict(test_)
accuracy_score(label_t, DT_expost_preds)



0.8706031830948241

In [27]:

print("gini decision tree classifier ")

print(confusion_matrix(label_t, DT_expost_preds))
print(classification_report(label_t, DT_expost_preds))

gini decision tree classifier 
[[9747  812]
 [1123 3272]]
              precision    recall  f1-score   support

           0       0.90      0.92      0.91     10559
           1       0.80      0.74      0.77      4395

    accuracy                           0.87     14954
   macro avg       0.85      0.83      0.84     14954
weighted avg       0.87      0.87      0.87     14954



In [28]:
dtree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)
dtree.fit(train_, label_)

coreml_model = sklearn.convert(dtree, key_words, 'output')
coreml_model.save('coreml_dtree.mlmodel')

pred = dtree.predict(test_)
accuracy_score(label_t, pred)


0.8706031830948241

In [29]:
model = coremltools.models.MLModel('coreml_dtree.mlmodel')
# model.predict({'input': temp_test_data})
pred_out_put = model.predict({'account':42.11758, 'member':19.31017, 'access':17.05149, 'email':16.6749, 'address':15.3276,
             'update':15.3276, 'price':12.55556, 'market':10.23963, 'online':9.770984, 'information':9.474589,
             'work':8.331161, 'credit':5.883471, 'response':4.504742, 'offer':3.3573, 'transaction':2.809165,
             'agreement':1.863823, 'registration':1.791679, 'person':1.522086, 'system':1.276965,
             'process':1.076627, 'service':0.895862, 'request':0.616316, 'message':0.32805})
print(pred_out_put["output"])
pred_out_put
# accuracy_score(temp_test_labels,dt_predictions)

0


{'output': 0,
 'classProbability': {0: 0.9951219512195122, 1: 0.004878048780487805}}

In [30]:
print("entropy decision tree classifier ")

print(confusion_matrix(label_t, pred))
print(classification_report(label_t, pred))

entropy decision tree classifier 
[[9747  812]
 [1123 3272]]
              precision    recall  f1-score   support

           0       0.90      0.92      0.91     10559
           1       0.80      0.74      0.77      4395

    accuracy                           0.87     14954
   macro avg       0.85      0.83      0.84     14954
weighted avg       0.87      0.87      0.87     14954



In [31]:
# from sklearn.svm import SVC
# only linearSVC can be converted 
from sklearn.svm import LinearSVC as SVC
# svc=SVC(kernel='sigmoid')
svc=SVC()
inputs = np.asarray(train_, dtype=float)
outputs = np.asarray(label_, dtype=int)
model_5=svc.fit(inputs, outputs)

coreml_model = sklearn.convert(model_5, key_words, 'output')
coreml_model.save('coreml_svm.mlmodel')

pred = model_5.predict(test_)
accuracy_score(label_t,pred)


/Users/lidayuan/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8555570415942223

In [32]:
model = coremltools.models.MLModel('coreml_svm.mlmodel')
# model.predict({'input': temp_test_data})
pred_out_put = model.predict({'account':42.11758, 'member':19.31017, 'access':17.05149, 'email':16.6749, 'address':15.3276,
             'update':15.3276, 'price':12.55556, 'market':10.23963, 'online':9.770984, 'information':9.474589,
             'work':8.331161, 'credit':5.883471, 'response':4.504742, 'offer':3.3573, 'transaction':2.809165,
             'agreement':1.863823, 'registration':1.791679, 'person':1.522086, 'system':1.276965,
             'process':1.076627, 'service':0.895862, 'request':0.616316, 'message':0.32805})
print(pred_out_put["output"])
pred_out_put

1


{'output': 1,
 'classProbability': {0: 0.45708471741666756, 1: 0.5429152825833324}}

In [33]:
print("svm ")

print(confusion_matrix(label_t,pred))
print(classification_report(label_t,pred))


svm 
[[10120   439]
 [ 1721  2674]]
              precision    recall  f1-score   support

           0       0.85      0.96      0.90     10559
           1       0.86      0.61      0.71      4395

    accuracy                           0.86     14954
   macro avg       0.86      0.78      0.81     14954
weighted avg       0.86      0.86      0.85     14954



In [34]:
from sklearn.ensemble import AdaBoostClassifier
adc=AdaBoostClassifier(n_estimators=5,learning_rate=1)
model_6=adc.fit(train_, label_)

pred = model_6.predict(test_)
accuracy_score(label_t,pred)


0.8599037046943961

In [35]:
print("AdaBoostClassifier ")

print(confusion_matrix(label_t,pred))
print(classification_report(label_t,pred))

AdaBoostClassifier 
[[10066   493]
 [ 1602  2793]]
              precision    recall  f1-score   support

           0       0.86      0.95      0.91     10559
           1       0.85      0.64      0.73      4395

    accuracy                           0.86     14954
   macro avg       0.86      0.79      0.82     14954
weighted avg       0.86      0.86      0.85     14954



In [36]:
import numpy as np
from xgboost import XGBClassifier

xgb=XGBClassifier()
# model_7=xgb.fit(temp_train_data, temp_train_label)
# pred = model_7.predict(temp_test_data)
model_7=xgb.fit(np.asarray(train_, dtype=float), np.asarray(label_, dtype=int))
pred = model_7.predict(np.asarray(test_))


accuracy_score(label_t,pred)


0.8896616289955864

In [37]:
print("xgboost ")

print(confusion_matrix(label_t, pred))
print(classification_report(label_t, pred))

xgboost 
[[10017   542]
 [ 1108  3287]]
              precision    recall  f1-score   support

           0       0.90      0.95      0.92     10559
           1       0.86      0.75      0.80      4395

    accuracy                           0.89     14954
   macro avg       0.88      0.85      0.86     14954
weighted avg       0.89      0.89      0.89     14954



In [38]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier()
model_gb=gb.fit(train_, label_)

# coreml_model = sklearn.convert(model_gb, key_words, 'output')
# coreml_model.save('coreml_gb.mlmodel')

pred = model_gb.predict(test_)
accuracy_score(label_t, pred)


0.891199679015648

In [39]:
print("AdaBoostClassifier ")

print(confusion_matrix(label_t,pred))
print(classification_report(label_t, pred))


AdaBoostClassifier 
[[10028   531]
 [ 1096  3299]]
              precision    recall  f1-score   support

           0       0.90      0.95      0.92     10559
           1       0.86      0.75      0.80      4395

    accuracy                           0.89     14954
   macro avg       0.88      0.85      0.86     14954
weighted avg       0.89      0.89      0.89     14954



"../test_data/from_weicheng/normal_txt/from your dvr's snap jpg.txt"

In [3]:
#test method for normal

import os
files = []

for dirname, _, filenames in os.walk('../test_data/from_weicheng/normal_txt'):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))
files[0]

"../test_data/from_weicheng/normal_txt/from your dvr's snap jpg.txt"

In [4]:
def get_data_content(files):
    data = []
    for file in files:
        with open(file, 'r') as f:
            data.append(f.read())
    return data

file_contents = get_data_content(files)
len(file_contents)

20

In [5]:
import coremltools
from coremltools.converters import sklearn

num = 0
for i in file_contents:
    feature_data = MailContentsFeaturize(i).run()
    model = coremltools.models.MLModel('coreml_dt.mlmodel')
# model.predict({'input': temp_test_data})
    pred_out_put = model.predict(feature_data)
    if str(pred_out_put["output"]) != '0':
        # print(i)
        print(files[num])
    print(pred_out_put["output"])
    num = num + 1


0
0
../test_data/from_weicheng/normal_txt/[Legal Notice] Data Processing and Security Terms update to address the invalidation of EU-US Privacy Shield.txt
1
0
../test_data/from_weicheng/normal_txt/zendesk.txt
1
0
0
0
../test_data/from_weicheng/normal_txt/Invitation_ Ask an Expert_ Oceans @ Wed May 6, 2020 11am - 11_30am (PDT) (zhangshuhao@gmail.com).txt
1
0
../test_data/from_weicheng/normal_txt/Security alert for your linked Google Account.txt
1
../test_data/from_weicheng/normal_txt/3.txt
1
0
0
../test_data/from_weicheng/normal_txt/Security alert.txt
1
../test_data/from_weicheng/normal_txt/calendar-google.txt
1
../test_data/from_weicheng/normal_txt/security-alert.txt
1
0
0
../test_data/from_weicheng/normal_txt/[k7pa.com] Email configuration settings for “edisontesting@k7pa.com”.txt
1


In [7]:
#test method for phishing

import os
files = []

for dirname, _, filenames in os.walk('../test_data/from_weicheng/phishing_txt'):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))
files[0]

'../test_data/from_weicheng/phishing_txt/Edison Mail in the of the Best Email Apps for Android.txt'

In [8]:
def get_data_content(files):
    data = []
    for file in files:
        with open(file, 'r') as f:
            data.append(f.read())
    return data

file_contents = get_data_content(files)
len(file_contents)

16

In [9]:
num = 0
for i in file_contents:
    feature_data = MailContentsFeaturize(i).run()
    model = coremltools.models.MLModel('coreml_dt.mlmodel')
# model.predict({'input': temp_test_data})
    pred_out_put = model.predict(feature_data)
    print(pred_out_put["output"])
    if str(pred_out_put["output"]) != '1':
        # print(i)
        print(files[num])
    num = num + 1


1
0
../test_data/from_weicheng/phishing_txt/CSAC Alarm for 2019-10-01 20_00_00 at Home.txt
1
0
../test_data/from_weicheng/phishing_txt/neteasy.txt
1
0
../test_data/from_weicheng/phishing_txt/E-Ticket Confirmation-BRUEFC 08DEC.txt
1
0
../test_data/from_weicheng/phishing_txt/hotmail.txt
0
../test_data/from_weicheng/phishing_txt/4.txt
0
../test_data/from_weicheng/phishing_txt/[Email-macOS] Feedback from brian.walker@walkercriminallawyers.com.au.txt
0
../test_data/from_weicheng/phishing_txt/I will forward the video to your relatives, friends, and co workers.txt
0
../test_data/from_weicheng/phishing_txt/Earn a $150  cash rewards bonus when you spend $500  on purchases in the first 3 months.txt
0
../test_data/from_weicheng/phishing_txt/File _Copy  of July Bonus 30JULY2020.xls_  has been shared with you..txt
1
0
../test_data/from_weicheng/phishing_txt/ticket-180841.txt
0
../test_data/from_weicheng/phishing_txt/ComputerBase Downloads_ Nvidia PhysX System Software 9.19.0218 (Deutsch).txt
